In [20]:
# Install necessary packages
!pip install transformers datasets torchaudio librosa pandas
!pip install scikit-fuzzy
!pip install ahpy
!pip install PyDrive

In [21]:
# Import necessary libraries
import os
import torch
import gc
import librosa
import pandas as pd
import numpy as np
import math
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from ahpy import Compare
from google.colab import files, drive, auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from torch.utils.data import Dataset
from transformers import (
    AutoModelForAudioClassification, Wav2Vec2FeatureExtractor,
    AutoTokenizer, MobileBertForSequenceClassification,
    BigBirdForSequenceClassification, Trainer, TrainingArguments,
    DataCollatorWithPadding
)
import joblib
import torch.nn.functional as F

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
# ---------------------------------------
# Emotion Recognition Module
# ---------------------------------------

class PretrainedEmotionModel:
    """
    Use the pre-trained DistilHuBERT model from Hugging Face for emotion classification.
    """
    def __init__(self, model_name):
        # Load the pre-trained DistilHuBERT model and feature extractor
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.model = AutoModelForAudioClassification.from_pretrained(model_name)

    def predict_all_labels(self, audio_file_path):
        """
        Predict all possible emotion labels with their respective confidence scores.
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load and preprocess the audio
        speech_array, sampling_rate = librosa.load(audio_file_path, sr=16000)
        inputs = self.feature_extractor(
            speech_array, return_tensors="pt", sampling_rate=sampling_rate, padding=True
        )

        # Move inputs and model to the appropriate device (GPU or CPU)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        self.model.to(device)

        # Make predictions
        with torch.no_grad():
            logits = self.model(**inputs).logits
            probabilities = torch.softmax(logits, dim=-1).squeeze().cpu().numpy()

        # Map label IDs to emotion labels
        id2label = self.model.config.id2label

        # Collect the predictions and their associated confidence scores
        results = []
        for label_id, confidence in enumerate(probabilities):
            emotion = id2label[label_id]
            results.append({
                'audio_file': os.path.basename(audio_file_path),
                'emotion': emotion.capitalize(),
                'confidence': confidence
            })

        # Sort results by confidence in descending order
        df = pd.DataFrame(results)
        df = df.sort_values(by='confidence', ascending=False).reset_index(drop=True)

        return df


class EmotionPipeline:
    """
    Orchestrates the workflow of downloading audio data from Google Drive and performing emotion classification.
    """
    def __init__(self, model_name="pollner/distilhubert-finetuned-ravdess"):
        self.model_name = model_name
        self.model = PretrainedEmotionModel(self.model_name)

    def authenticate_and_create_drive(self):
        """
        Authenticates the user and creates a PyDrive GoogleDrive instance.
        """
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)
        return drive

    def download_audio_from_drive(self, drive, audio_file_id, destination_path):
        """
        Downloads an audio file from Google Drive using its file ID.
        """
        print(f"Downloading audio file from Google Drive with file ID: {audio_file_id}")
        downloaded = drive.CreateFile({'id': audio_file_id})
        downloaded.GetContentFile(destination_path)
        print(f"Downloaded audio file and saved as {destination_path}")

    def load_and_predict(self, audio_file_ids):
        """
        Downloads the audio files using their file IDs and predicts all possible labels.
        """
        drive = self.authenticate_and_create_drive()

        for audio_file_name, audio_file_id in audio_file_ids.items():
            destination_path = f"./{audio_file_name}"
            self.download_audio_from_drive(drive, audio_file_id, destination_path)

            # Perform prediction using the pre-trained model
            result_df = self.model.predict_all_labels(destination_path)
        return result_df

In [23]:
# ---------------------------------------
# Sentiment Analysis Module
# ---------------------------------------

class TextSentimentAnalysisPipeline:
    def __init__(self, dataset_path, model_name='cambridgeltl/sst_mobilebert-uncased'):
        with open(dataset_path, 'r') as file:
            self.conversation = [line.strip() for line in file.readlines()]
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = MobileBertForSequenceClassification.from_pretrained(model_name)
        self.model.eval()  # Set model to evaluation mode

    def extract_caller_text(self):
        # Extract lines spoken by the "Caller"
        caller_lines = [line.lstrip('"Caller: ').rstrip('", ').split('. ') for line in self.conversation if line.startswith('"Caller:')]
        caller_lines = [sentence for sublist in caller_lines for sentence in sublist]
        return caller_lines

    def predict_sentiments(self, texts):
        inputs = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
        outputs = self.model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return probs.detach().numpy()

    def evaluate(self):
        # Extract all text spoken by the Caller
        caller_text = self.extract_caller_text()

        # Predict sentiments and get confidence scores
        result = []
        for turn in caller_text:
          prob = self.predict_sentiments(turn)
          result.append({
              'Sentence': turn,
              'Positive Confidence': prob[0, 2],
              'Negative Confidence': prob[0, 0],
              'Neutral Confidence': prob[0, 1]
        })
        result_df = pd.DataFrame(result)
        avg_pos = result_df['Positive Confidence'].mean()
        avg_neg = result_df['Negative Confidence'].mean()
        avg_neu = result_df['Neutral Confidence'].mean()
        sentiment_df = pd.DataFrame({'sentiment':['Positive', 'Negative', 'Neutral'], 'confidence':[avg_pos, avg_neg, avg_neu]})
        return sentiment_df

In [24]:
# ---------------------------------------
# Coherence Evaluation Module
# ---------------------------------------

# Step 1: Class for defining the custom dataset
class DialogueDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        context = self.dataframe.iloc[idx, 0]
        response = self.dataframe.iloc[idx, 1]
        label = self.dataframe.iloc[idx, 2]

        combined_text = context + " " + self.tokenizer.sep_token + " " + response
        encoding = self.tokenizer(
            combined_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label, dtype=torch.long),
        }

# Step 2: Class for model training
class ModelTrainer:
    def __init__(self, train_dataset):
        self.tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-base')
        self.model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base')
        self.train_dataset = train_dataset
        self.training_args = self._setup_training_args()
        for name, param in self.model.named_parameters():
          if not param.is_contiguous():
            #print(f'Making contiguous:{name}')
            param.data = param.data.contiguous()
        #for name, param in self.model.named_parameters():
            #print(f'Layer:{name}, Contiguous:{param.is_contiguous()}')

    def _setup_training_args(self):
        # Set up training arguments, limiting to 1 epoch for quick testing
        return TrainingArguments(
            output_dir='./results',
            num_train_epochs=1,  # Quick testing with 1 epoch
            per_device_train_batch_size=8,
            learning_rate=2e-5,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=50,
            save_total_limit=2,
            save_steps=200,
            evaluation_strategy="no",
        )

    def fine_tune_model(self):
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset
        )
        trainer.train()
        return self.model

    def save_model(self, save_path):
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
        print(f"Model saved to {save_path}")

# Step 3: Class for coherence evaluation
class CoherenceEvaluator:
    def __init__(self, model_path):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = BigBirdForSequenceClassification.from_pretrained(model_path)

    def tokenize_input(self, context, response):
        return self.tokenizer(context, response, return_tensors='pt', max_length=1024, truncation=True, padding='max_length')

    def compute_logits(self, inputs):
        outputs = self.model(**inputs)
        return outputs.logits

    def apply_softmax(self, logits):
        probabilities = F.softmax(logits, dim=1)
        return probabilities[0][1].item()

# Step 4: Main pipeline class to encapsulate the entire process
class CoherencePipeline:
    def __init__(self, dataset_path, model_save_path, load_model=False):
        self.file_path = dataset_path
        self.model_save_path = model_save_path
        self.load_model = load_model
        self.model_trainer = None
        self.coherence_evaluator = None

    def prepare_dataset(self):
        df = pd.read_csv(self.file_path)
        tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-base')
        train_dataset = DialogueDataset(df, tokenizer, max_length=256)
        return train_dataset

    def train_and_save_model(self, train_dataset):
        self.model_trainer = ModelTrainer(train_dataset)
        trained_model = self.model_trainer.fine_tune_model()
        self.model_trainer.save_model(self.model_save_path)
        return trained_model

    def evaluate_coherence(self):
        #file_name = list(self.file_path.keys())[0]
        with open(self.file_path, 'r') as file:
            dialogue = file.readlines()

        self.coherence_evaluator = CoherenceEvaluator(self.model_save_path)
        pairs = [(dialogue[i].strip(), dialogue[i + 1].strip()) for i in range(len(dialogue) - 1)]

        scores = []
        for context, response in pairs:
            inputs = self.coherence_evaluator.tokenize_input(context, response)
            logits = self.coherence_evaluator.compute_logits(inputs)
            score = self.coherence_evaluator.apply_softmax(logits)
            scores.append(score)

        # Create DataFrame to store results
        df_results = pd.DataFrame({
            'Pair Number': [f'Pair {i+1}' for i in range(len(pairs))],
            'Context': [pair[0] for pair in pairs],
            'Response': [pair[1] for pair in pairs],
            'Coherence Score': scores
        })

        # Calculate overall coherence score
        overall_score = sum(scores) / len(scores)
        df_results.loc['Overall'] = ['', '', 'Overall Coherence Score', overall_score]

        return df_results

    def run_pipeline(self):
        if self.load_model:
            # Check if fine-tuned model exists
            if self.model_save_path.startswith('google/'):
              print(f'Using pretrained model from Hugging Face:{self.model_save_path}')
            else:
                if not os.path.exists(self.model_save_path):
                    raise FileNotFoundError(f"No fine-tuned model found at {self.model_save_path}. Please train the model first.")
                print(f"Using existing model from {self.model_save_path}")
        else:
            # Train model if flag is set to True
            train_dataset = self.prepare_dataset()
            self.train_and_save_model(train_dataset)

        # Proceed to evaluate test data
        df_results = self.evaluate_coherence()
        print(df_results)
        return df_results

In [25]:
# ---------------------------------------
# Utility Functions Module
# ---------------------------------------

def map_emotion_to_score(emotion):
    """
    Maps emotion labels to numerical scores.
    """
    emotion_scores = {
        'Happy': 1,
        'Neutral': 0,
        'Calm': 0,
        'Angry': -1,
        'Disgust': -1,
        'Surprised': -1,
        'Fearful': -1,
        'Sad': -1
    }
    return emotion_scores.get(emotion, 0)

def map_sentiment_to_score(sentiment):
    """
    Maps sentiment labels to numerical scores.
    """
    sentiment_scores = {'Neutral': 0, 'Negative': -1, 'Positive': 1}
    return sentiment_scores.get(sentiment, 0)

def sigmoid_transform(x):
    """
    Applies a sigmoid transformation and scales the result.
    """
    x_sigmoid = 1 / (1 + math.exp(-x))
    x_scaled = x_sigmoid * 10
    return x_scaled

def calculate_weighted_score(df):
    """
    Calculates the final weighted score from a DataFrame containing emotion or sentiment data.
    """
    if 'emotion' in df.columns:
        # Process DataFrame with emotions
        df['score'] = df['emotion'].apply(map_emotion_to_score)
        df['weighted_score'] = df['score'] * df['confidence']
    elif 'sentiment' in df.columns:
        # Process DataFrame with sentiments
        df['score'] = df['sentiment'].apply(map_sentiment_to_score)
        df['weighted_score'] = df['score'] * df['confidence']
    else:
        raise ValueError("DataFrame format not recognized.")

    weighted_sum = df['weighted_score'].sum()
    total_confidence = df['confidence'].sum()
    weighted_score_raw = weighted_sum / total_confidence if total_confidence != 0 else 0
    weighted_score = sigmoid_transform(weighted_score_raw)
    return weighted_score

In [38]:
# ---------------------------------------
# AHP and Fuzzy Logic Module
# ---------------------------------------

class ScoringSystem:
    """
    Implements AHP weight calculation, consistency ratio verification, and fuzzy logic scoring.
    """
    def __init__(self):
        self.weights = None

    def calculate_ahp_weights(self):
        """
        Calculates AHP weights and checks consistency ratio.
        """
        # AHP Weight Calculation
        criteria = ['Emotion Score', 'Sentiment Score', 'Coherence Score']
        pairwise_comparisons = {
            ('Emotion Score', 'Sentiment Score'): 2,  # Emotion Score is slightly important than Sentiment Score
            ('Emotion Score', 'Coherence Score'): 3,  # Emotion Score is moderate important than Coherence Score
            ('Sentiment Score', 'Coherence Score'): 3  # Sentiment Score is moderate important than Coherence Score
        }

        comparison = Compare('Criteria', pairwise_comparisons, precision=3, random_index='saaty')
        report = comparison.report(show=True)

        self.weights = report['target_weights']
        consistency_ratio = report['elements']['consistency_ratio']

        # Check Consistency Ratio (CR) to ensure AHP consistency
        if consistency_ratio > 0.1:
            raise ValueError(f"Consistency Ratio (CR) is too high: {consistency_ratio:.3f}. The pairwise comparisons may be inconsistent.")
        else:
            print(f"Consistency Ratio (CR): {consistency_ratio:.3f} (acceptable)")

        print("AHP Weights:", self.weights)

    def perform_fuzzy_logic_scoring(self, emotion_score, sentiment_score, coherence_score):
        """
        Performs fuzzy logic scoring using the calculated AHP weights.
        """
        # Define fuzzy variables and membership functions
        x_range = np.arange(0, 10.1, 0.1)
        emotion_score_fuzzy = ctrl.Antecedent(x_range, 'emotion_score')
        sentiment_score_fuzzy = ctrl.Antecedent(x_range, 'sentiment_score')
        coherence_score_fuzzy = ctrl.Antecedent(x_range, 'coherence_score')
        final_score_fuzzy = ctrl.Consequent(x_range, 'final_score')

        # Define membership functions for input variables
        for fuzzy_var in [emotion_score_fuzzy, sentiment_score_fuzzy, coherence_score_fuzzy]:
          fuzzy_var['low'] = fuzz.trimf(fuzzy_var.universe, [0, 0, 1.5])
          fuzzy_var['medium'] = fuzz.trimf(fuzzy_var.universe, [1, 3, 6])
          fuzzy_var['high'] = fuzz.trimf(fuzzy_var.universe, [5, 10, 10])

        # Define membership functions for the output variable
        final_score_fuzzy['low'] = fuzz.trimf(final_score_fuzzy.universe, [0, 0, 1.5])
        final_score_fuzzy['medium'] = fuzz.trimf(final_score_fuzzy.universe, [1, 3, 6])
        final_score_fuzzy['high'] = fuzz.trimf(final_score_fuzzy.universe, [5, 10, 10])

        # Define fuzzy rules
        # Rule 1: If all indicators are high, then the final score is high.
        rule1 = ctrl.Rule(
            emotion_score_fuzzy['high'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['high'],
            final_score_fuzzy['high']
        )

        # Rule 2: If two indicators are high and one is medium, then the final score is high.
        rule2 = ctrl.Rule(
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['high']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['high']),
            final_score_fuzzy['high']
        )

        # Rule 3: If two indicators are high and one is low, then the final score is medium.
        rule3 = ctrl.Rule(
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['low']) |
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['high']) |
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['high']),
            final_score_fuzzy['medium']
        )

        # Rule 4: If all indicators are medium, then the final score is medium.
        rule4 = ctrl.Rule(
            emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['medium'],
            final_score_fuzzy['medium']
        )

        # Rule 5: If one indicator is high and two are medium, then the final score is medium.
        rule5 = ctrl.Rule(
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['high']),
            final_score_fuzzy['medium']
        )

        # Rule 6: If two indicators are medium and one is low, then the final score is medium.
        rule6 = ctrl.Rule(
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['low']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['medium']),
            final_score_fuzzy['medium']
        )

        # Rule 7: If all indicators are low, then the final score is low.
        rule7 = ctrl.Rule(
            emotion_score_fuzzy['low'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['low'],
            final_score_fuzzy['low']
        )

        # Rule 8: If two indicators are low and one is medium, then the final score is low.
        rule8 = ctrl.Rule(
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['low']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['low']),
            final_score_fuzzy['low']
        )

        # Rule 9: If one indicator is high, one medium, and one low, then the final score is medium.
        rule9 = ctrl.Rule(
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['low']) |
            (emotion_score_fuzzy['high'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['low']) |
            (emotion_score_fuzzy['medium'] & sentiment_score_fuzzy['low'] & coherence_score_fuzzy['high']) |
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['high'] & coherence_score_fuzzy['medium']) |
            (emotion_score_fuzzy['low'] & sentiment_score_fuzzy['medium'] & coherence_score_fuzzy['high']),
            final_score_fuzzy['medium']
        )

        # Create control system and simulation
        scoring_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
        scoring = ctrl.ControlSystemSimulation(scoring_ctrl)

        # Calculate weighted scores
        weighted_emotion_score = emotion_score * self.weights['Emotion Score']
        weighted_sentiment_score = sentiment_score * self.weights['Sentiment Score']
        weighted_coherence_score = coherence_score * self.weights['Coherence Score']

        print(f"Weighted Emotion Score: {weighted_emotion_score:.2f}")
        print(f"Weighted Sentiment Score: {weighted_sentiment_score:.2f}")
        print(f"Weighted Coherence Score: {weighted_coherence_score:.2f}")

        # Input weighted scores into fuzzy system
        scoring.input['emotion_score'] = weighted_emotion_score
        scoring.input['sentiment_score'] = weighted_sentiment_score
        scoring.input['coherence_score'] = weighted_coherence_score

        # Perform fuzzy inference
        scoring.compute()

        # Output final score
        final_score_value_no_fuzzy = weighted_emotion_score + weighted_sentiment_score + weighted_coherence_score
        final_score_value = scoring.output['final_score']
        return final_score_value_no_fuzzy, final_score_value

In [39]:
# ---------------------------------------
# Main Function
# ---------------------------------------

def main():
    # ---------------------------
    # Emotion Recognition
    # ---------------------------
    audio_model_name = "pollner/distilhubert-finetuned-ravdess"

    # Initialize the pipeline with the pre-trained model
    emotion_pipeline = EmotionPipeline(model_name=audio_model_name)

    # Provide the Google Drive file IDs of the audio files
    audio_file_ids = {
        'audio1.mp3': '108kPpEQeA_6RkQXmmLWDJXQzdiISlm0r'
    }

    # Download the audio files and perform emotion prediction
    audio_results_df = emotion_pipeline.load_and_predict(audio_file_ids)
    print("Audio Emotion Results:")
    print(audio_results_df)

    # ---------------------------
    # Sentiment Analysis
    # ---------------------------
    test_data_path = './dialogue1.txt'  # Ensure this file exists
    sentiment_pipeline = TextSentimentAnalysisPipeline(test_data_path)
    text_results_df = sentiment_pipeline.evaluate()
    print("Text Sentiment Results:")
    print(text_results_df)

    # ---------------------------
    # Calculate Scores
    # ---------------------------
    # Calculate emotion and sentiment scores
    final_score_emotion = calculate_weighted_score(audio_results_df)
    final_score_sentiment = calculate_weighted_score(text_results_df)

    print(f"Emotion Score: {final_score_emotion:.2f}")
    print(f"Sentiment Score: {final_score_sentiment:.2f}")

    # ---------------------------
    # Coherence Evaluation
    # ---------------------------
    coherence_pipeline = CoherencePipeline(
        dataset_path = test_data_path, # Change to '/content/dialogues_dataset.csv' if you want to train
        model_save_path='./coherence_model', load_model=True   # Set to False if you want to train
        )
    coherence_result = coherence_pipeline.run_pipeline()
    coherence_score_raw = coherence_result.loc['Overall', 'Coherence Score']
    coherence_score = coherence_score_raw * 10  # Assuming the raw score is between 0 and 1
    print(f"Coherence Score: {coherence_score:.2f}")

    # ---------------------------
    # AHP and Fuzzy Logic Scoring
    # ---------------------------
    scoring_system = ScoringSystem()
    scoring_system.calculate_ahp_weights()
    final_score_value_no_fuzzy, final_score_value = scoring_system.perform_fuzzy_logic_scoring(
        final_score_emotion, final_score_sentiment, coherence_score
    )
    print(f"Final Score (without Fuzzy Logic): {final_score_value_no_fuzzy:.2f}")
    print(f"Final Score (Defuzzified): {final_score_value:.2f}")

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at pollner/distilhubert-finetuned-ravdess were not used when initializing HubertForSequenceClassification: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at pollner/distilhubert-finetuned-ravdess and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_em

Downloaded audio file and saved as ./audio1.mp3
Audio Emotion Results:
   audio_file    emotion  confidence
0  audio1.mp3    Neutral    0.606273
1  audio1.mp3        Sad    0.158475
2  audio1.mp3      Happy    0.151102
3  audio1.mp3       Calm    0.070715
4  audio1.mp3    Disgust    0.005959
5  audio1.mp3      Angry    0.003035
6  audio1.mp3  Surprised    0.002574
7  audio1.mp3    Fearful    0.001867


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Text Sentiment Results:
  sentiment  confidence
0  Positive    0.415585
1  Negative    0.193286
2   Neutral    0.391129
Emotion Score: 4.95
Sentiment Score: 5.55
Using existing model from ./coherence_model
        Pair Number                                            Context  \
0            Pair 1  ["AI: Hi, my name is Lila. I'm Octivo's AI age...   
1            Pair 2  "Caller: Hey, nice to meet you. My name is Mic...   
2            Pair 3  "AI: Thank you for introducing yourself Michae...   
3            Pair 4  "Caller: Yeah, sure. I'm 27 but I feel like I ...   
4            Pair 5  "AI: I completely understand your hesitation a...   
5            Pair 6  "Caller: Ok, that's fair enough. So I'm earnin...   
6            Pair 7  "AI: Thank you for sharing your income range t...   
7            Pair 8  "Caller: I will retire at around 65 and I woul...   
Overall                                                                  

                                                  Res